In [86]:
import torch
import json

In [87]:
with open('/home/remote_user/sharefile/AIoT_code/time_series_model/data/MaxMinData.json', 'r') as file:
    data = json.load(file)
PDmax = data['PDmax']
PDmin = data['PDmin']
Tmax = data['Tmax']
Tmin = data['Tmin']
ACCmax = data['ACCmax']
ACCmin = data['ACCmin']

In [88]:
PDmax, PDmin, Tmax, Tmin, ACCmax, ACCmin

(10.831600000000071, 0.0015199999999822467, 30.5, 25.0, 32.0, 21.0)

In [89]:
def temp_normalize(T):
    return ((T - Tmin)/(Tmax - Tmin))
        
def PD_normalize(PD):    
    return ((PD - PDmin) / (PDmax - PDmin))

def ACC_normalize(ACC):
    return ((ACC - ACCmin) / (ACCmax - ACCmin))

def temp_denormalize(T):
    return ((Tmax - Tmin) * T + Tmin)
        
def PD_denormalize(PD):    
    return ((PDmax - PDmin) * PD + PDmin)

def ACC_denormalize(ACC):
    return ((ACCmax - ACCmin) * ACC + ACCmin)

In [90]:
# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/PD_Model.pkl'
loaded_model = torch.load(path)
PD_model = loaded_model['model'].to(device)
PD_model.eval()

path = '/home/remote_user/sharefile/AIoT_code/time_series_model/model/Temp_Model.pkl'
loaded_model = torch.load(path)
Temp_model = loaded_model['model'].to(device)
Temp_model.eval()

LSTM(
  (lstm): LSTM(9, 32, batch_first=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [91]:
ori_data = torch.load('/home/remote_user/sharefile/AIoT_code/time_series_model/data/dataset.pt')
ori_data.shape

torch.Size([5716, 30, 9])

In [92]:
ori_data[100][0][0]

tensor(0.1818)

In [93]:
temp_denormalize(ori_data[100][0][0])

tensor(26.)

In [94]:
for i in range(0, 30, 10):
    print('室內溫度 :', round(temp_denormalize(ori_data[i][-1][0]).item(), 4))
    for j in [21,22,23,24,25,26,27,32]:
        jj = ACC_normalize(j)
        ori_data[i][-3][2] = jj
        ori_data[i][-2][2] = jj
        ori_data[i][-1][2] = jj
        res = Temp_model(ori_data[i].unsqueeze(0).to(device))
        res = temp_denormalize(res.item())
        print(j,"度:", round(res,4))
    print('='*20)
        

室內溫度 : 27.14
21 度: 27.1574
22 度: 27.1565
23 度: 27.1556
24 度: 27.1545
25 度: 27.1533
26 度: 27.152
27 度: 27.1506
32 度: 27.142
室內溫度 : 26.74
21 度: 26.7746
22 度: 26.7722
23 度: 26.7697
24 度: 26.7671
25 度: 26.7644
26 度: 26.7617
27 度: 26.7588
32 度: 26.7431
室內溫度 : 26.36
21 度: 26.403
22 度: 26.3996
23 度: 26.3961
24 度: 26.3925
25 度: 26.3888
26 度: 26.385
27 度: 26.3812
32 度: 26.3608


In [95]:
import torch

# 創建二維 Tensor
tensor = torch.tensor([[1, 2, 3, 4],
                       [5, 6, 7, 8],
                       [9, 10, 11, 12]])

# 提取範圍 [1][0] 到 [1][3] 的所有數據
extracted_data = tensor[1, 0:4]

# 打印結果
print(extracted_data)


tensor([5, 6, 7, 8])
